In [74]:
import os
import re
import netCDF4 as nc
import xarray as xr
import pandas as pd
import geopandas as gpd

from apollo import hydropoint

In [61]:
# Get all the shapefiles
shp_files = [file for file in os.listdir() if file.endswith('.shp')]

for shp_file in shp_files:
    gdf = gpd.read_file(shp_file)
    gdf = gdf.set_geometry('geometry')
    display(gdf)
    print(gdf.columns)

,geometry
0,"POLYGON ((516025.000 253525.000, 516025.000 25..."


Index(['geometry'], dtype='object')


,geometry
0,"POLYGON ((384500.000 227875.000, 384425.000 22..."


Index(['geometry'], dtype='object')


,geometry
0,"POLYGON ((282575.000 833550.000, 282575.000 83..."


Index(['geometry'], dtype='object')


In [62]:
import pandas as pd
flow = pd.read_csv('7001_gdf.csv')
flow = flow.drop(flow.columns[2], axis=1)
flow = flow.drop(flow.index[0:19])
flow.columns = ['Date', 'Flow']
flow

,Date,Flow
19,1960-10-01,3.934
20,1960-10-02,4.585
21,1960-10-03,15.820
22,1960-10-04,13.130
23,1960-10-05,7.925
...,...,...
22659,2022-09-26,4.342
22660,2022-09-27,18.910
22661,2022-09-28,16.920
22662,2022-09-29,8.032


In [68]:
import importlib
importlib.reload(hydropoint)

<module 'apollo.hydropoint' from 'C:\\Users\\aline\\soil-moisture-proxies\\apollo\\hydropoint.py'>

In [69]:
df = pd.DataFrame({
    'variable': range(10)
})

# Add shifted columns
df = hydropoint.weather_shift(df, 'variable', 3, past=True)
print(df)

   variable  variable-1  variable-2
0         0         NaN         NaN
1         1         0.0         NaN
2         2         1.0         0.0
3         3         2.0         1.0
4         4         3.0         2.0
5         5         4.0         3.0
6         6         5.0         4.0
7         7         6.0         5.0
8         8         7.0         6.0
9         9         8.0         7.0


In [10]:
file_path = 'Rainfall_1980_aggregated.nc'

ds = xr.open_dataset(file_path)
ds

<xarray.Dataset>
Dimensions:    (time: 366, latitude: 49, longitude: 49)
Coordinates:
  * time       (time) datetime64[ns] 1980-01-01 1980-01-02 ... 1980-12-31
  * longitude  (longitude) float32 -8.0 -7.75 -7.5 -7.25 ... 3.25 3.5 3.75 4.0
  * latitude   (latitude) float32 60.0 59.75 59.5 59.25 ... 48.5 48.25 48.0
Data variables:
    tp         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-04-14 12:05:10 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [72]:
domain_weather = xr.open_mfdataset(['Rainfall.nc',
                                    'Pressure.nc'])
surface_data = xr.open_dataset('Soil_Moisture.nc')

In [91]:
importlib.reload(hydropoint)

<module 'apollo.hydropoint' from 'C:\\Users\\aline\\soil-moisture-proxies\\apollo\\hydropoint.py'>

In [92]:
db = pd.read_csv('Catchment_Database.csv')
for i in range(len(db)):
    test = hydropoint.hydrobase(db.loc[i][0], db.loc[i][3], db.loc[i][4])
print(test)

In [93]:
days = 28
outdf = test.flow_meteorolgy_combine(domain_weather, surface_data, days)
#outpath = str(self.station) + '_lumped.csv'
#outdf.to_csv(outpath, index=True)

In [94]:
outdf

,Date,Flow,Temperature,U Windspeed,V Windspeed,Humidity,Rain,Resultant Windspeed,Rain-1,Rain-2,...,Humidity-25,Humidity-26,Humidity-27,Humidity_28_Mu,Humidity_90_Mu,Humidity_180_Mu,Soil Moisture 1,Soil Moisture 2,Soil Moisture 3,Soil Moisture 4
179,1979-06-29,9.654,282.597930,4.351510,0.944776,83.392270,4.181380,4.452891,0.889576,0.100817,...,64.267386,56.077831,47.931048,69.844041,68.247992,75.545634,0.254181,0.243434,0.219750,0.276735
180,1979-06-30,9.730,281.037340,3.683683,-2.257421,65.197733,0.687574,4.320355,4.181380,0.889576,...,95.441514,64.267386,56.077831,70.460708,68.130294,75.436394,0.246692,0.243917,0.219100,0.276375
181,1979-07-01,5.846,284.745192,4.448323,-0.333201,69.357661,0.717311,4.460785,0.687574,4.181380,...,50.771454,95.441514,64.267386,70.934988,68.203607,75.346883,0.235974,0.239829,0.218229,0.275998
182,1979-07-02,4.232,285.430261,4.511554,1.384902,78.912655,0.100297,4.719330,0.717311,0.687574,...,69.790186,50.771454,95.441514,71.458033,68.137718,75.310331,0.230940,0.236197,0.217307,0.275635
183,1979-07-03,3.607,288.045636,2.799352,0.458578,65.002569,0.010104,2.836665,0.100297,0.717311,...,82.045223,69.790186,50.771454,70.370928,68.128275,75.141033,0.219441,0.230287,0.215837,0.275231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15701,2021-12-27,6.629,278.783492,-0.496029,1.618723,95.908229,1.710861,1.693018,0.198059,0.037088,...,80.461919,87.395166,98.857538,89.743645,84.737521,78.166030,0.271568,0.266287,0.282178,0.286019
15702,2021-12-28,8.270,278.013985,0.233542,0.743381,100.357239,0.321579,0.779203,1.710861,0.198059,...,88.843708,80.461919,87.395166,89.797205,85.120314,78.481938,0.262474,0.264933,0.281010,0.285985
15703,2021-12-29,27.620,279.755516,-2.302575,2.796058,91.132859,8.542511,3.622125,0.321579,1.710861,...,100.360694,88.843708,80.461919,89.930694,85.148566,78.690262,0.311028,0.263023,0.279705,0.285942
15704,2021-12-30,40.830,285.346902,3.526217,3.450856,92.228023,3.634520,4.933823,8.542511,0.321579,...,85.343484,100.360694,88.843708,90.350912,85.317745,78.807801,0.316017,0.309548,0.282402,0.285897


In [90]:
from apollo import mechanics as ma

weather = test.meteorological_extraction(domain_weather)
surface = test.meteorological_extraction(surface_data)
weather = hydropoint.rename(weather)
surface = hydropoint.rename(surface)


weather['Rain'] = weather['Rain']*1000*24
weather['Date'] = pd.to_datetime(weather['Date'],
                                         format='%Y-%m-%d').dt.date
surface['Date'] = pd.to_datetime(surface['Date'],
                                         format='%Y-%m-%d').dt.date

surface = surface.groupby('Date').first().reset_index()
weather = weather.groupby('Date').first().reset_index()

weather = weather.drop(['longitude', 'latitude'], axis=1)
surface = surface.drop(['longitude', 'latitude'], axis=1)

weather['Resultant Windspeed'] = (weather['U Windspeed']**2
                                          + weather['V Windspeed']**2)**(1/2)
for f in ['Rain','Temperature','Resultant Windspeed','Humidity']:
    weather = hydropoint.weather_shift(weather, f, days)
    for window in [28, 90, 180]:
        ma.stat_roller(weather, f, window, method='mean')

'''

combined = pd.merge(weather, surface, on='Date')
combined = pd.merge(self.flow, combined, how='inner', on='Date')
combined = combined.drop(combined.index[0:179])
'''
weather

,Date,Temperature,U Windspeed,V Windspeed,Humidity,Rain,Resultant Windspeed,Rain-1,Rain-2,Rain-3,...,Humidity-21,Humidity-22,Humidity-23,Humidity-24,Humidity-25,Humidity-26,Humidity-27,Humidity_28_Mu,Humidity_90_Mu,Humidity_180_Mu
0,1979-01-01,271.576250,2.304902,0.946802,84.860900,0.782791,2.491788,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1979-01-02,272.984203,-0.102852,1.894139,85.469579,1.253943,1.896930,0.782791,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1979-01-03,275.197394,0.185399,3.487820,85.492037,0.597700,3.492744,1.253943,0.782791,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1979-01-04,273.443249,-0.037133,-0.553775,95.476273,8.640998,0.555019,0.597700,1.253943,0.782791,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1979-01-05,270.880672,0.684582,2.238401,93.179481,1.960725,2.340746,8.640998,0.597700,1.253943,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15701,2021-12-27,278.783492,-0.496029,1.618723,95.908229,1.710861,1.693018,0.198059,0.037088,3.132288,...,86.057290,85.343484,100.360694,88.843708,80.461919,87.395166,98.857538,89.743645,84.737521,78.166030
15702,2021-12-28,278.013985,0.233542,0.743381,100.357239,0.321579,0.779203,1.710861,0.198059,0.037088,...,74.293444,86.057290,85.343484,100.360694,88.843708,80.461919,87.395166,89.797205,85.120314,78.481938
15703,2021-12-29,279.755516,-2.302575,2.796058,91.132859,8.542511,3.622125,0.321579,1.710861,0.198059,...,98.922584,74.293444,86.057290,85.343484,100.360694,88.843708,80.461919,89.930694,85.148566,78.690262
15704,2021-12-30,285.346902,3.526217,3.450856,92.228023,3.634520,4.933823,8.542511,0.321579,1.710861,...,93.453139,98.922584,74.293444,86.057290,85.343484,100.360694,88.843708,90.350912,85.317745,78.807801
